In [0]:
#!import "../Utils/Queries"
#!import "../Utils/ImportCalculationMethods"
#!import "../Report/ReportConfigurationAndUtils"
#!import "TestData"

# Initialization

In [0]:
await DataSource.UpdateAsync( new[] { partitionReportingNode } );
await DataSource.UpdateAsync( new[] { partition, previousPeriodPartition, partitionScenarioMTUP, previousPeriodPartitionScenarioMTUP } );
await DataSource.UpdateAsync( new[] { dt11 } );

# Import Queries: Raw Variables

## Data definition

In [0]:
var bestEstimateRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = new[] {1.0} }, 
                                  new RawVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = new[] {2.0} },
                                  new RawVariable { AmountType = "PR", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = new[] {3.0} },
                                  new RawVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = new[] {4.0} },
                                  new RawVariable { AmountType = "CL", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = new[] {5.0} },
                                  new RawVariable { AmountType = "CL", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = new[] {6.0} } };

In [0]:
var previousScenarioRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = previousPeriodPartitionScenarioMTUP.Id, Values = new[] {3.15} }, 
                                      new RawVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = previousPeriodPartitionScenarioMTUP.Id, Values = new[] {7.17} } };

In [0]:
var scenarioRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {1.1} }, 
                              new RawVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {2.1} } };

In [0]:
var newScenarioRawVars = new[] { new RawVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {11.0} }, 
                                 new RawVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = new[] {41.0} } };

## Scenario Query and Workspace with new scenario

Raw variable query test. <br>
Scenario cashflow re-import. <br>
DataSource with BestEstimate and Scenario raw variable, Workspace with new raw variable for Scenario. <br>
 

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
await ws.UpdateAsync(newScenarioRawVars);

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(52.0);

## Scenario query, Workspace with no scenario, mismatching format

Raw variable query test. <br>
Scenario Actual import. <br>
DataSource with BestEstimate and Scenario raw variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(0);

## Scenario query, Workspace with no scenario and DataSource with scenario

Raw variable query test. <br>
Scenario parameter (re-)import. <br>
DataSource with BestEstimate and Scenario raw variable, Workspace without any raw variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(scenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(3.2);

## Scenario query, Workspace with no scenario and DataSource with only Best Estimate

Raw variable query test. <br>
Scenario parameter (re-)import. <br>
DataSource with BestEstimate raw variable, Workspace without any raw variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(21);

## Scenario query, Workspace with no scenario and DataSource with Best Estimate and previous period scenario

Raw variable query test. <br>
Scenario parameter (re-)import. <br>
DataSource with BestEstimate and previous period Scenario raw variable, Workspace without any raw variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<RawVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateRawVars.Concat(previousScenarioRawVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedRawVars = await ws.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedRawVars.SelectMany(x => x.Values).Sum().Should().Be(21);

# Import Queries: Ifrs Variables

## Data definition

In [0]:
var bestEstimateIfrsVars = new[] { new IfrsVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(1.0) }, 
                                   new IfrsVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(2.0) },
                                   new IfrsVariable { AmountType = "PR", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(3.0)},
                                   new IfrsVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(4.0)},
                                   new IfrsVariable { AmountType = "CL", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(5.0)},
                                   new IfrsVariable { AmountType = "CL", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(6.0)} };

In [0]:
var previousScenarioIfrsVars = new[] { new IfrsVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = previousPeriodPartitionScenarioMTUP.Id, Values = SetProjectionValue(3.15) }, 
                                       new IfrsVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = previousPeriodPartitionScenarioMTUP.Id, Values = SetProjectionValue(7.17) } };

In [0]:
var scenarioIfrsVars = new[] { new IfrsVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(1.1) }, 
                               new IfrsVariable { AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(2.1) } };

In [0]:
var newScenarioIfrsVars = new[] { new IfrsVariable { AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(11.0) }, 
                                  new IfrsVariable { AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(41.0) } };

## Scenario Query and Workspace with new scenario

Ifrs variable query test. <br>
Scenario Actual re-import. <br>
DataSource with BestEstimate and Scenario ifrs variable, Workspace with new ifrs variable for Scenario. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateIfrsVars.Concat(scenarioIfrsVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
await ws.UpdateAsync(newScenarioIfrsVars);

In [0]:
var queriedIfrsVars = await ws.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedIfrsVars.Select(x => x.Values.Sum()).Sum().Should().Be(52.0);

## Scenario query, Workspace with no scenario, mismatching format

Ifrs variable query test. <br>
Scenario parameter (re-)import. <br>
DataSource with BestEstimate and Scenario ifrs variable, Workspace without any ifrs variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateIfrsVars.Concat(scenarioIfrsVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedIfrsVars = await ws.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Cashflow);

In [0]:
queriedIfrsVars.Select(x => x.Values.Sum()).Sum().Should().Be(0);

## Scenario query, Workspace with no scenario and DataSource with scenario

Ifrs variable query test. <br>
Scenario Actual import of an empty file. <br>
DataSource with BestEstimate and Scenario ifrs variable, Workspace without any ifrs variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateIfrsVars.Concat(scenarioIfrsVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedIfrsVars = await ws.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedIfrsVars.Select(x => x.Values.Sum()).Sum().Should().Be(3.2);

## Scenario query, Workspace with no new scenario and DataSource with only Best Estimate

Ifrs variable query test. <br>
Scenario Actual import of an empty file. <br>
DataSource with BestEstimate ifrs variable, Workspace without any ifrs variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateIfrsVars.ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedIfrsVars = await ws.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedIfrsVars.Select(x => x.Values.Sum()).Sum().Should().Be(3.2);

## Scenario query, Workspace with no scenario and DataSource with Best Estimate and previous period scenario

Ifrs variable query test. <br>
Scenario Actual import of an empty file. <br>
DataSource with BestEstimate and previous period Scenario ifrs variable, Workspace without any ifrs variable. <br>

In [0]:
await DataSource.DeleteAsync( await DataSource.Query<IfrsVariable>().ToArrayAsync() );
await DataSource.UpdateAsync( bestEstimateIfrsVars.Concat(previousScenarioIfrsVars).ToArray() );
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
var queriedIfrsVars = await ws.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(DataSource, partitionScenarioMTUP.Id, partition.Id, ImportFormats.Actual);

In [0]:
queriedIfrsVars.Select(x => x.Values.Sum()).Sum().Should().Be(21);

# Report

## Data

In [0]:
var bestEstimateIfrsVars = new[] { new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(1.0) }, 
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(2.0) },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(3.0) },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "CL", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(4.0) },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "AU", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(5.0) },
                                   new IfrsVariable { DataNode = "DT1.1", AmountType = "CL", AocType = "EV", Novelty = "I", Partition = partition.Id, Values = SetProjectionValue(6.0) } };

In [0]:
var scenarioIfrsVars = new[] { new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "CL", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(1.1) }, 
                               new IfrsVariable { DataNode = "DT1.1", AmountType = "PR", AocType = "AU", Novelty = "I", Partition = partitionScenarioMTUP.Id, Values = SetProjectionValue(2.1) } };

## Best estimate query

Report variable query test. <br>
Report for Best Estimate ( or Scenario Delta and Scenario All). <br>
DataSource and Workspace with BestEstimate and Scenario ifrs variable. <br>

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await ws.UpdateAsync(bestEstimateIfrsVars);
await ws.UpdateAsync(scenarioIfrsVars);

In [0]:
(await ws.QueryReportVariablesAsync((args.Year, args.Month, args.ReportingNode, args.Scenario))).Select(x => x.Value).Sum().Should().Be(21);

## Scenario query

Report variable query test. <br>
Report for Scenario. <br>
DataSource and Workspace with BestEstimate and Scenario ifrs variable. <br>

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await ws.UpdateAsync(bestEstimateIfrsVars);
await ws.UpdateAsync(scenarioIfrsVars);

In [0]:
(await ws.QueryReportVariablesAsync((args.Year, args.Month, args.ReportingNode, argsScenarioMTUP.Scenario))).Select(x => x.Value).Sum().Should().Be(21.2);